In [ ]:
import pandas as pd
import time
import random
import math
import mmh3
from statistics import mean

# Teste de tempo de permancia dos blocos: usando descarte

In [ ]:
def str_to_MinHash(str1, q, seed=0):
    return min([mmh3.hash(str1[i:i + q], seed) for i in range(len(str1) - q + 1)])

def frequent2(temp, L, t):
    return {k: v for (k, v) in temp.items() if v/L >= t}

def matching():
    global tp, fp, pairsNo, L1, q
    for index2 in range(nbS, nbS + offsetB): 
        if index2 > len(df2) - 1:
            return True

        rr = df2.iloc[index2, 0:5]
        idScholar = rr["id"]
        title = rr["title"]
        authors = rr["authors"]
        srec = title + " " + authors

        temp = dict()
        indices = [random.randrange(0, L) for i in range(L1)]
        matchingPairs = {}
        for l in indices:
            key = str(str_to_MinHash(srec.lower(), q, l))
            d = dictB[l]
            if key in d:
                ids = d[key]
                for id in ids:
                    if id in temp:
                        temp[id] += 1
                        if temp[id] / L1 >= t:
                            matchingPairs[id] = 1
                    else:
                        temp[id] = 1
        for id in matchingPairs.keys():
            idDBLP = id
            pairsNo += 1
            if idDBLP in truthD:
                ids = truthD[idDBLP]
                for id in ids:
                    if id == idScholar:
                        tp += 1
                        break
            else:
                fp += 1

    return False

def elimina_elementos_dentro_dictB(array_para_descarte, array_para_descarte_igual, elementos_para_descarte):
    indice = 0
    for (tempo_inserido) in array_para_descarte_igual:
        if tempo_inserido < elementos_para_descarte:
            indice += 1
        else:
            del array_para_descarte[:indice]
            del array_para_descarte_igual[:indice]
            break

def normaliza_para_0(array,elementoAtualParaDescarte):
    return [e - elementoAtualParaDescarte for e in array]

In [ ]:
df1 = pd.read_csv("../00-datasets/DBLP.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("../00-datasets/Scholar.csv", sep=",", encoding="utf-8", keep_default_na=False)
truth = pd.read_csv("../00-datasets/truth.csv", sep=",", encoding="utf-8", keep_default_na=False)
truthD = dict()
for i, r in truth.iterrows():
    idDBLP = r["idDBLP"]
    idScholar = r["idScholar"]
    if idDBLP in truthD:
        ids = truthD[idDBLP]
        ids.append(idScholar)
    else:
        truthD[idDBLP] = [idScholar]

t = 0.5
TP = 5347
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0

In [ ]:
tempoQueFoiInseridoNaEstrutura = 0 # esse é o valor que sera colocado junto com o id, seria a posiçaõ do elemento no array
elementos_para_descarte = 50 # esse é a quantidade de elementos que serão descartados
acompanhamentoIndicePorBloco = {} #essa estrutura é um indice que vai ser usada para saber qual a posição do elemento que será descartado

tempo_permanencia = []
while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:5]
        idDBLP = rr["id"]
        title = rr["title"]
        authors = rr["authors"]

        srec = title + " " + authors
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                ids = d[key]
                ids_igual = d_igual[key]
                if len(ids) < w:
                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    qtd_descarte, elementoAtualParaDescarte = acompanhamentoIndicePorBloco[(key, l)]
                    
                    array_normalizado = normaliza_para_0(ids_igual, elementoAtualParaDescarte - 50)
                    print(elementoAtualParaDescarte)
                    print(ids_igual)
                    print(array_normalizado)
                    print("\n\n\n")
                    media = mean(array_normalizado)
                    tempo_permanencia.append(media)

                    elimina_elementos_dentro_dictB(ids, ids_igual, elementoAtualParaDescarte)
                    acompanhamentoIndicePorBloco[(key, l)] = [qtd_descarte + 1, elementoAtualParaDescarte + 50]

                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
            else:
                d[key] = [idDBLP]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]
                acompanhamentoIndicePorBloco[(key, l)] = [0, 50]

        tempoQueFoiInseridoNaEstrutura += 1
    
    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    # termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    # if termination:
    #         break
    if naS >= len(df1):
        break
    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

In [ ]:
mean(tempo_permanencia)
#979.4007080291971

# Teste de tempo de permancia dos blocos: usando pop

In [ ]:
df1 = pd.read_csv("../00-datasets/DBLP.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("../00-datasets/Scholar.csv", sep=",", encoding="utf-8", keep_default_na=False)
truth = pd.read_csv("../00-datasets/truth.csv", sep=",", encoding="utf-8", keep_default_na=False)
truthD = dict()
for i, r in truth.iterrows():
    idDBLP = r["idDBLP"]
    idScholar = r["idScholar"]
    if idDBLP in truthD:
        ids = truthD[idDBLP]
        ids.append(idScholar)
    else:
        truthD[idDBLP] = [idScholar]

t = 0.5
TP = 5347
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0

In [ ]:
tempoQueFoiInseridoNaEstrutura = 0  # Contador de inserção
elementos_para_descarte = 1  # Quantidade de elementos descartados
acompanhamentoIndicePorBloco = {}  # Índice para rastrear descartes

tempo_permanencia = []  # Lista para armazenar os tempos de permanência

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:5]
        idDBLP = rr["id"]
        title = rr["title"]
        authors = rr["authors"]

        srec = title + " " + authors
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                ids = d[key]
                ids_igual = d_igual[key]
                if len(ids) < w:
                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    qtd_descarte, elementoAtualParaDescarte = acompanhamentoIndicePorBloco[(key, l)]
                    print(key)
                    print(elementoAtualParaDescarte - elementos_para_descarte)
                    print(ids_igual)                   
                    array_normalizado = normaliza_para_0(ids_igual, elementoAtualParaDescarte - elementos_para_descarte)
                    print(array_normalizado)
                    print("\n\n\n")
                    media = mean(array_normalizado)
                    tempo_permanencia.append(media)
            
                    ids.pop(0)
                    ids_igual.pop(0)
                    # elimina_elementos_dentro_dictB(ids, ids_igual, elementoAtualParaDescarte)
                    acompanhamentoIndicePorBloco[(key, l)] = [qtd_descarte + 1, elementoAtualParaDescarte + elementos_para_descarte]

                    ids.append(idDBLP)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)

            else:
                # Cria uma nova entrada no dicionário
                d[key] = [idDBLP]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]
                acompanhamentoIndicePorBloco[(key, l)] = [0, elementos_para_descarte]

        tempoQueFoiInseridoNaEstrutura += 1
    
    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    # termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    # if termination:
    #     break
    
    if naS >= len(df1):
        break

    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

In [ ]:
mean(tempo_permanencia)
#1132.0064994299928